[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.mx)

# Ejemplo de uso con *JSON* y *Jinja*.

En este capítulo se creará una aplicación que realice una búsqueda de datos:

* Regresando un objeto serializado en formato *JSON*.
* Desplegando una página *HTML* mediante el uso de plantillas de *Jinja*.

## La función ```enuentra()```.

```
encuentra(<patron>, <obj>, <iterable>
```

In [ ]:
encuentra = lambda patron, obj, iterable: bool(sum([patron.casefold() \
            in obj[item].casefold() for item in iterable]))

**Ejemplos:**

In [ ]:
encuentra('nez', 
          {'nombre': 'Juan',
           'apellido': 'Godínez',
           'correo': 'falsonez@falso.com'},
          ['apellido', 'nombre'])

In [ ]:
encuentra('Juan',
          {'nombre': 'Juan',
           'apellido': 'Godínez',
           'correo': 'falsonez@falso.com'},
          ['correo'])

In [ ]:
encuentra('n', 
          {'nombre': 'Juan',
           'apellido': 'Godínez',
           'correo': 'falsonez@falso.com'},
          ['correo', 'nombre', 'apellido'])

## La función ```buscar_archivo()```.

```
buscar_archivo(<patron>, <ruta>, <iterable>
```

In [ ]:
def buscar_archivo(patron, ruta, iterable):
    with open(ruta, 'tr') as f:
        base = eval(f.read())
    return [item for item in base if encuentra(patron, item, iterable)]

**Ejemplo:**

El archivo [```data/alumnos.txt```](data/alumnos.txt) contiene la representación de un objeto tipo ```list``` que a su vez contiene una sucesión de objetos tipo ```dict``` con los identificadores:

* ```'cuenta'```
* ```'nombre'```
* ```'primer_apellido'```
* ```'segundo_apellido'```
* ```'carrera'```
* ```'semestre'```
* ```'promedio'```
* ```'al_corriente'```.

```
[{'cuenta': 1231221, 'al_corriente': False, 'carrera': 'Arquitectura', 'nombre': 'Pedro', 'primer_apellido': 'Solis', 'promedio': 7.8, 'semestre': 3, 'segundo_apellido': 'Cabañas'},
{'cuenta': 1231222, 'al_corriente': False, 'carrera': 'Actuaría', 'nombre': 'Yolanda', 'primer_apellido': 'Jiménez', 'segundo_apellido': 'Lerdo', 'promedio': 6, 'semestre': 3},
{'cuenta': 1231223, 'al_corriente': True, 'carrera': 'sistemas', 'nombre': 'Juan', 'primer_apellido': 'Ramos', 'segundo_apellido': 'Breña', 'promedio': 8.6, 'semestre': 9},
{'cuenta': 1231224, 'al_corriente': True, 'carrera': 'Derecho', 'nombre': 'Mayra Jimena', 'primer_apellido': 'Cervantes', 'segundo_apellido': 'Lisama', 'promedio': 9.2, 'semestre': 12}]
```
* Se utilizará la función ```buscar_archivo()``` para encontrar los elementos que coincidan con la cadena de búsqueda en los campos ```'nombre'```, ```'primer_apellido'``` y ```'segundo_apellido'``` dentro del archivo ```data/alumnos.txt```.

In [ ]:
campos = ('nombre', 'primer_apellido', 'segundo_apellido')
ruta = 'data/alumnos.txt'

In [ ]:
buscar_archivo('ed', ruta, campos)

In [ ]:
buscar_archivo('z', ruta, campos)

In [ ]:
buscar_archivo('Derecho', ruta, campos)

In [ ]:
buscar_archivo('Ramos', ruta, campos)

## Aplicación de búsqueda.

La siguiente aplicación levantará un servicio web que regresará una búsqueda con la función ```buscar_archivo()``` si la cadena de búsqueda se ingresa como parte de la ruta.

* Si la ruta es ```/api/<cadena>```regresará el resultado de la búsqueda de ```<cadena>```  en formato *JSON*.

* Si la ruta es ```/despliega/<cadena>``` regresará un documento *HTML* desplegando el resultado de la búsqueda.

### Definición de la plantilla.

La plantilla [```templates/busqueda.html```](templates/busqueda.html), que contiene el siguiente código que desplegará los datos de cada elemento encontrado y su contenido es el siguiente.

``` html
<h1> Alumnos Encontrados:</h1>
<ul>
    {% for alumno in alumnos %}
    <li>{% for campo in ['nombre', 'primer_apellido', 'segundo_apellido'] %}
{{alumno[campo]}}
        {% endfor %}</li> 
    {% endfor %}
</ul>
```

### El código principal.

In [ ]:
import json
from flask import Flask, jsonify, render_template

app =Flask(__name__)


@app.route('/api/<termino>')
def busqueda(termino):
    '''Regresa el resultado de la búsqueda en formato JSON'''
    return(jsonify(buscar_archivo(str(termino), ruta, campos)))


@app.route('/despliega/<termino>')
def despliega(termino):
    '''Despliega el resultado de la búsqueda en un documento HTML'''
    return render_template('busqueda.html',
                           alumnos=buscar_archivo(str(termino),
                           ruta,
                           campos))

http://localhost:5000/api/Ramos

http://localhost:5000/despliega/Ramos

**Advertencia:** Una vez ejecutada la siguiente celda, es necesario interrumpir el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run(host="0.0.0.0", port=5000)

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>